# Notebook: Train Geocentric Residual Model

Trains the V2 Physics-Informed MLP.

In [ ]:
import sys
import os
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import StandardScaler

# Import local library from Parent Dir
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))
import stars_utils

print(f"TensorFlow Version: {tf.__version__}")

In [ ]:
data_path = '../mars_processed_data.csv'
df = pd.read_csv(data_path)

FEATURES = [
    'Time_Index', 'Time_Index_2', 
    'Sin_Mars', 'Cos_Mars',
    'Sin_Jupiter', 'Cos_Jupiter', 'Inv_Dist_Jupiter',
    'Sin_Saturn', 'Cos_Saturn', 'Inv_Dist_Saturn',
    'Sin_Venus', 'Cos_Venus', 'Inv_Dist_Venus',
    'X_au_Lag1', 'Y_au_Lag1', 'Z_au_Lag1', 
    'X_au_Lag2', 'Y_au_Lag2', 'Z_au_Lag2',
    'Kepler_X', 'Kepler_Y', 'Kepler_Z'     
]
TARGETS = ['Res_X', 'Res_Y', 'Res_Z']

X = df[FEATURES].values
y = df[TARGETS].values

TEST_SIZE = 0.2
split_index = int(len(df) * (1 - TEST_SIZE))
X_train, X_test = X[:split_index], X[split_index:]
y_train, y_test = y[:split_index], y[split_index:]

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

y_scaler = StandardScaler()
y_train_scaled = y_scaler.fit_transform(y_train)
y_test_scaled = y_scaler.transform(y_test)

print("Data Prepared.")

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(X_train_scaled.shape[1],)),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(3, activation='linear') 
])

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), loss='mse')
model.fit(
    X_train_scaled, y_train_scaled,
    validation_data=(X_test_scaled, y_test_scaled),
    epochs=500,
    batch_size=64,
    callbacks=[tf.keras.callbacks.EarlyStopping(patience=50, restore_best_weights=True)],
    verbose=1
)

In [ ]:
# Save Models to Root/models
model_dir = '../models'
os.makedirs(model_dir, exist_ok=True)

model.save(os.path.join(model_dir, 'mars_geocentric_v2.keras'))
stars_utils.save_scaler(scaler, os.path.join(model_dir, 'scaler_features.pkl'))
stars_utils.save_scaler(y_scaler, os.path.join(model_dir, 'scaler_targets.pkl'))

print("Model Artifacts saved to ../models/")